<a href="https://colab.research.google.com/github/danangcorp/sql-dqlab/blob/main/DANANG_ARYA_SAPUTRA_SQL___Number_Date_Format_%26_SubQuery_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b><span style='color:#0B2F9F'>Pengantar</span></b>

Dalam pengelolaan data, kemampuan untuk menampilkan informasi dengan format yang tepat serta menggali data secara mendalam menjadi hal yang sangat penting. Format angka dan tanggal membantu memastikan data tersaji secara konsisten, mudah dibaca, dan sesuai konteks, sehingga hasil analisis lebih akurat dan informatif. Sementara itu, penggunaan subquery memungkinkan pengambilan data secara lebih fleksibel dan efisien, dengan cara memecah proses pencarian menjadi langkah-langkah yang lebih terarah. Dengan memahami kedua konsep ini, seseorang dapat mengolah data dengan lebih rapi, melakukan analisis yang lebih mendalam, serta menghasilkan informasi yang lebih bermakna dan dapat diandalkan.


# <b><span style='color:#0B2F9F'>Set up</span></b>

Dibutuhkan proses autentikasi dari Google Colab ke Google Big Query. Ikuti langkah berikut https://drive.google.com/file/d/1gW8alZ_PrvcrsieqWCHOR4ssLI_25BRc/view untuk detail step-by-step nya

In [ ]:
# Import library yang dibutuhkan
from google.colab import auth, data_table
from google.cloud import bigquery
from pandas_gbq import to_gbq

# Proses autentikasi akun
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
# Buat BigQuery client
project_id = 'nezaproject-123456'
client = bigquery.Client(project = project_id)

Sebagai Data Engineer, Antara (Jr. Data Engineer di DQPizza) ditugaskan untuk membuat data mart untuk kebutuhan detail transaksi. Dengan memanfaatkan relasi antar table, Antara mengeksekusi suatu perintah  untuk membuat table f_detail_transaksi (sebuah tabel fact yang berisi data transaksi secara rinci, mencakup ukuran-ukuran kuantitatif seperti jumlah, harga, total nilai transaksi, dan lain sebagainya) hingga terbentuklah tabel f_detail_transaksi.

<br>
<code>SELECT * FROM dqlab-9876543.dq_pizza.f_detail_transaksi</code>
<br><br>
<img src="https://raw.githubusercontent.com/bachtiyarma/Material/main/Image/Materi-SQL/SQL%20-%20f_detail_transaksi.png" width="30%">

Data dibawah (f_detail_transaksi) akan digunakan pada modul ini

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
  *
FROM `dqlab-9876543.dq_pizza.f_detail_transaksi`

Executing query with job ID: a4de580e-a00a-49be-981d-4c04975066d4
Query executing: 1.43s
Job ID a4de580e-a00a-49be-981d-4c04975066d4 successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,order_id,order_details_id,customer_id,order_maker_id,pizza_name,category,size,price,production_cost,quantity,order_date,order_time,completion_time,is_complain,complain_detail
0,TRX000734,TRXD001645,CUST003549,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,01/12/2024,20:00:30,20:27:49,0,None
1,TRX004103,TRXD009366,CUST006714,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,03/10/2024,19:10:04,19:39:33,0,None
2,TRX001914,TRXD004305,CUST008674,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,02/01/2024,19:24:03,19:36:58,0,None
3,TRX017344,TRXD039387,CUST004020,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,10/22/2024,13:20:05,13:39:42,0,None
4,TRX019817,TRXD045082,CUST003570,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,12/03/2024,16:40:44,17:07:19,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48618,TRX016165,TRXD036640,CUST005956,EMP000756,THE BRIE CARRE PIZZA,supreme,S,191565.0,170505.0,1,09/29/2024,21:05:44,21:33:24,0,None
48619,TRX008475,TRXD019280,CUST004817,EMP000756,THE BRIE CARRE PIZZA,supreme,S,191565.0,170505.0,1,05/22/2024,16:56:23,17:20:01,0,None
48620,TRX001581,TRXD003564,CUST009381,EMP000756,THE GREEK PIZZA,CLASSIC,XXL,291195.0,260738.0,1,01/27/2024,16:28:38,16:54:32,0,None
48621,TRX016753,TRXD037944,CUST004945,EMP000756,THE GREEK PIZZA,CLASSIC,XXL,291195.0,260738.0,1,10/10/2024,17:54:55,18:06:01,0,None


# <b>A. <span style='color:#0B2F9F'><code>Data Type</code></span></b>

Data type dalam BigQuery adalah cara untuk mendefinisikan jenis data yang akan disimpan dalam tabel. Setiap kolom dalam tabel memiliki tipe data tertentu, yang menentukan apa yang dapat disimpan di kolom tersebut dan bagaimana data tersebut dapat diproses. Memahami tipe data ini penting untuk desain skema yang efisien dan untuk memastikan bahwa kueri dapat dijalankan dengan optimal.

<br>
<img src="https://hackolade.com/help/lib/BigQuery%20data%20types%20table.png" width="40%">

docs : <i><a href="https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types">https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types</a></i>

# <b>B. <span style='color:#0B2F9F'><code>Operasi Tipe Data Numerik</code></span></b>

Dalam SQL, tipe data numerik digunakan untuk menyimpan nilai angka dan dapat dikelompokkan menjadi beberapa jenis sesuai peruntukannya (INT, FLOAT, NUMERIC, dll). Selain itu pada tipe data Numerik juga dapat dilakukan operasi matematis diantaranya :
<ol>
    <li>Operasi Aritmatika</li>
    <li>Fungsi Matematis</li>
    <li>Agregasi</li>
</ol>

## <b>B.1. <span style='color:#0B2F9F'><code>Operasi Aritmatika</code></span></b>
SQL mendukung operasi dasar aritmatika seperti penjumlahan, pengurangan, perkalian, pembagian dan modulo.

<img src="https://mazamanbd.wordpress.com/wp-content/uploads/2014/09/dd.jpg"><br><br>

Untuk dapat menerapkannya pada sebuah tabel gunakan contoh format berikut

<img src="https://raw.githubusercontent.com/bachtiyarma/Material/refs/heads/main/Image/Materi-SQL/SQL%20-%20OperasiAritmatika.png" width="50%">

<i>docs : <a href="https://cloud.google.com/bigquery/docs/reference/standard-sql/operators">https://cloud.google.com/bigquery/docs/reference/standard-sql/operators</a></i>

#### <i><b><span style='color:#55679C'>Quest</span> : Pada tabel f_detail_transaksi, tampilkan semua kolom dan tambahkan satu kolom 'total_price' yang merupakan hasil perkalian antara harga dengan kuantitas (terbeli)! Urutkan berdasarkan kuantitas paling besar hingga ke terkecil </b></i>

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
  *,
  (price * quantity) AS total_price
FROM `dqlab-9876543.dq_pizza.f_detail_transaksi`
ORDER BY quantity DESC

Executing query with job ID: 3f7803bb-d478-4ef3-9e6e-44cc6db8373b
Query executing: 1.51s
Job ID 3f7803bb-d478-4ef3-9e6e-44cc6db8373b successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,order_id,order_details_id,customer_id,order_maker_id,pizza_name,category,size,price,production_cost,quantity,order_date,order_time,completion_time,is_complain,complain_detail,total_price
0,TRX005264,TRXD011977,CUST004837,EMP000286,THE BIG MEAT PIZZA,Classic,S,97200.0,81567.0,4,03/30/2024,13:24:35,13:35:43,0,None,388800.0
1,TRX008286,TRXD018876,CUST004105,EMP000437,THE BIG MEAT PIZZA,Classic,S,97200.0,81567.0,4,05/19/2024,12:42:46,13:11:40,0,None,388800.0
2,TRX015674,TRXD035497,CUST006814,EMP000302,THE CALIFORNIA CHICKEN PIZZA,CHICKEN,L,168075.0,118989.0,4,09/19/2024,14:16:35,14:40:53,0,None,672300.0
3,TRX018845,TRXD042847,CUST007150,EMP000756,THE BARBECUE CHICKEN PIZZA,CHICKEN,M,135675.0,99468.0,3,11/18/2024,12:25:12,12:50:38,0,None,407025.0
4,TRX000144,TRXD000342,CUST006964,EMP000231,THE BARBECUE CHICKEN PIZZA,CHICKEN,M,135675.0,99468.0,3,01/03/2024,13:44:28,14:06:42,0,None,407025.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48618,TRX018442,TRXD041913,CUST007150,EMP000437,THE MEDITERRANEAN PIZZA,VEGGIE,S,97200.0,74844.0,1,11/10/2024,19:49:13,20:02:01,0,None,97200.0
48619,TRX006536,None,CUST001704,EMP000665,None,None,None,NaN,NaN,<NA>,04/20/2024,16:04:24,16:26:08,0,None,NaN
48620,TRX006537,None,CUST006034,EMP000560,None,None,None,NaN,NaN,<NA>,04/20/2024,16:39:18,17:02:44,0,None,NaN
48621,TRX006505,None,CUST001704,EMP000560,None,None,None,NaN,NaN,<NA>,04/20/2024,11:59:36,12:11:41,0,None,NaN


#### <i><b><span style='color:#55679C'>Quest</span> : Selain total price, tambahkan juga total profit yang didapat tiap transaksi! (Lanjutkan hasil query quest diatas) </b></i>

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT *,
(price * quantity) AS total_price,
((price - production_cost) * quantity) AS total_profit
FROM `dqlab-9876543.dq_pizza.f_detail_transaksi`
ORDER BY quantity DESC;

Executing query with job ID: 702212d1-c299-4f90-89aa-1264ee85d147
Query executing: 1.50s
Job ID 702212d1-c299-4f90-89aa-1264ee85d147 successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,order_id,order_details_id,customer_id,order_maker_id,pizza_name,category,size,price,production_cost,quantity,order_date,order_time,completion_time,is_complain,complain_detail,total_price,total_profit
0,TRX005264,TRXD011977,CUST004837,EMP000286,THE BIG MEAT PIZZA,Classic,S,97200.0,81567.0,4,03/30/2024,13:24:35,13:35:43,0,None,388800.0,62532.0
1,TRX008286,TRXD018876,CUST004105,EMP000437,THE BIG MEAT PIZZA,Classic,S,97200.0,81567.0,4,05/19/2024,12:42:46,13:11:40,0,None,388800.0,62532.0
2,TRX015674,TRXD035497,CUST006814,EMP000302,THE CALIFORNIA CHICKEN PIZZA,CHICKEN,L,168075.0,118989.0,4,09/19/2024,14:16:35,14:40:53,0,None,672300.0,196344.0
3,TRX018845,TRXD042847,CUST007150,EMP000756,THE BARBECUE CHICKEN PIZZA,CHICKEN,M,135675.0,99468.0,3,11/18/2024,12:25:12,12:50:38,0,None,407025.0,108621.0
4,TRX000144,TRXD000342,CUST006964,EMP000231,THE BARBECUE CHICKEN PIZZA,CHICKEN,M,135675.0,99468.0,3,01/03/2024,13:44:28,14:06:42,0,None,407025.0,108621.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48618,TRX018442,TRXD041913,CUST007150,EMP000437,THE MEDITERRANEAN PIZZA,VEGGIE,S,97200.0,74844.0,1,11/10/2024,19:49:13,20:02:01,0,None,97200.0,22356.0
48619,TRX006536,None,CUST001704,EMP000665,None,None,None,NaN,NaN,<NA>,04/20/2024,16:04:24,16:26:08,0,None,NaN,NaN
48620,TRX006537,None,CUST006034,EMP000560,None,None,None,NaN,NaN,<NA>,04/20/2024,16:39:18,17:02:44,0,None,NaN,NaN
48621,TRX006505,None,CUST001704,EMP000560,None,None,None,NaN,NaN,<NA>,04/20/2024,11:59:36,12:11:41,0,None,NaN,NaN


#### <i><b><span style='color:#55679C'>Quest</span> : Bagaimana dengan customer paling loyal? Paling loyal diukur dari banyak profit restaurant yang dihasilkan. Tampilkan customer_id, total_frequency (berapa kali pembelian dalam setahun) dan total_profit yang diperoleh perusahaan, kelompokkan tiap customer_id! Urutkan berdasarkan customer_id dengan profit paling tinggi ke paling rendah! </b></i>

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
  customer_id,
  COUNT(DISTINCT order_id) AS total_frequency,
  SUM((price-production_cost) * quantity) AS total_profit
FROM `dqlab-9876543.dq_pizza.f_detail_transaksi`
GROUP BY customer_id
ORDER BY total_profit DESC;

Executing query with job ID: 6080e9bd-04b2-4568-aaa7-a669ac176d29
Query executing: 0.42s
Job ID 6080e9bd-04b2-4568-aaa7-a669ac176d29 successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,customer_id,total_frequency,total_profit
0,CUST006333,741,47622169.0
1,CUST009381,702,44017426.0
2,CUST007656,614,37358334.0
3,CUST003549,580,36914292.0
4,CUST006964,521,32351562.0
...,...,...,...
115,CUST008078,21,1070496.0
116,CUST005536,24,987714.0
117,CUST006892,12,858681.0
118,CUST003871,13,685908.0


## <b>B.2. <span style='color:#0B2F9F'><code>Fungsi Matematis</code></span></b>
SQL juga mendukung fungsi matematis dasar yang disajikan pada tabel berikut :

<table border="1">
    <thead>
        <tr>
            <th>Fungsi</th>
            <th>Deskripsi</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td><code>ABS()</code></td>
            <td>Mengambil nilai absolut</td>
        </tr>
        <tr>
            <td><code>ROUND()</code></td>
            <td>Membulatkan angka</td>
        </tr>
        <tr>
            <td><code>CEIL()</code></td>
            <td>Membulatkan ke atas</td>
        </tr>
        <tr>
            <td><code>FLOOR()</code></td>
            <td>Membulatkan ke bawah</td>
        </tr>
        <tr>
            <td><code>POW()</code></td>
            <td>Menghitung pangkat</td>
        </tr>
    </tbody>
</table>

Untuk dapat menerapkannya pada sebuah tabel gunakan contoh format berikut

<img src="https://raw.githubusercontent.com/bachtiyarma/Material/refs/heads/main/Image/Materi-SQL/SQL-FungsiMatematis.png" width="50%">

<i>docs : <a href="https://cloud.google.com/bigquery/docs/reference/standard-sql/mathematical_functions">https://cloud.google.com/bigquery/docs/reference/standard-sql/mathematical_functions</a></i>

#### <i><b><span style='color:#55679C'>Quest</span> : Sendja (Data Analyst di DQPizza) baru menyadari bahwa profit yang ia hitung pada quest sebelumnya merupakan profit sebelum pajak. Asumsikan pajak yang perlu dikeluarkan adalah 11% tiap kali transaksi. Hitung profit_before_tax, profit_after_tax dan tax_paid tiap order_id dan customer_id! Lakukan perhitungan untuk setiap transaksi yang valid (order_details_id tidak kosong)!</b></i>

Note : Bulatkan profit after tax sebanyak 2 angka dibelakang koma agar lebih rapi

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
  order_id,
  customer_id,
  ROUND(SUM(((price - production_cost) * quantity)),2) AS profit_before_tax,
  ROUND(SUM(((price - production_cost) * quantity)*(1-11/100)),2) AS profit_after_tax,
  SUM((((price - production_cost) * quantity) - ((price - production_cost) * quantity)*(1-11/100))) AS tax_paid
FROM `dqlab-9876543.dq_pizza.f_detail_transaksi`
where order_details_id IS NOT NULL
GROUP BY  order_id, customer_id


Executing query with job ID: a38c21ed-a343-41eb-af9b-b1eb615d8099
Query executing: 1.33s
Job ID a38c21ed-a343-41eb-af9b-b1eb615d8099 successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,order_id,customer_id,profit_before_tax,profit_after_tax,tax_paid
0,TRX004103,CUST006714,88209.0,78506.01,9702.99
1,TRX003947,CUST009755,16038.0,14273.82,1764.18
2,TRX018599,CUST001735,74520.0,66322.80,8197.20
3,TRX011935,CUST004354,74034.0,65890.26,8143.74
4,TRX000387,CUST004307,157950.0,140575.50,17374.50
...,...,...,...,...,...
21341,TRX014773,CUST009740,93069.0,82831.41,10237.59
21342,TRX003295,CUST009740,24705.0,21987.45,2717.55
21343,TRX002839,CUST009740,22761.0,20257.29,2503.71
21344,TRX003352,CUST009740,49329.0,43902.81,5426.19


# <b>C. <span style='color:#0B2F9F'><code>Operasi Tipe Data Date</code></span></b>

Dalam SQL, tipe data date digunakan untuk menyimpan atau mengoperasikan data tanggal. Tipe data tanggal dapat dilakukan operasi berikut, diantaranya :
<ol>
    <li>Menampilkan Tanggal & Waktu saat ini</li>
    <li>Menambahkan atau Mengurangi Tanggal</li>
    <li>Mendapatkan Bagian Tanggal</li>
    <li>Menghitung Selisih Tanggal</li>
</ol>

<br><b>NOTE</b><br><br>
Hal-hal yang perlu diperhatikan dalam penggunaan operasi `date` adalah :
<ul>
    <li>Format tanggal standar SQL adalah 'YYYY-MM-DD' atau tahun - bulan - tanggal, contoh : 2024-10-14 menunjukan tanggal 14 October 2024</li>
    <li>Mengakses tanggal pada SQL perlu menggunakan tanda apit petik satu, contoh : '2024-10-14'</li>
    <li>Pastikan nilai atau kolom tanggal tipe datanya adalah DATE atau TIMESTAMP, jika tidak maka bisa dikonversi menjadi <code>DATE(tanggal)</code> atau <code>TIMESTAMP(tanggal)</code></li>
</ul>

<i>docs : <a href="https://cloud.google.com/bigquery/docs/reference/standard-sql/date_functions">https://cloud.google.com/bigquery/docs/reference/standard-sql/date_functions</a></i>

## <b>C.1. <span style='color:#0B2F9F'><code>Menampilkan Tanggal & Waktu Saat Ini</code></span></b>

Untuk menampilkan tanggal dan waktu saat ini dapat menggunakan fungsi berikut
<table border="1">
    <thead>
        <tr>
            <th>Fungsi</th>
            <th>Deskripsi</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td><code>CURRENT_DATE()</code></td>
            <td>Mendapatkan Tanggal Hari ini Saja</td>
        </tr>
        <tr>
            <td><code>CURRENT_TIMESTAMP()</code></td>
            <td>Mendapatkan Tanggal & Waktu Hari ini (UTC)</td>
        </tr>
    </tbody>
</table>

UTC atau (Universal Time Coordinated) adalah waktu dasar yang tidak terpengaruh oleh zona waktu dan <i>daylight saving time</i>. UTC digunakan sebagai acuan untuk menghitung waktu di berbagai zona waktu di seluruh dunia (standar Greenwich Mean Time (London)). Selengkapnya baca <a href="https://internasional.kompas.com/read/2021/03/04/192042570/sejarah-penetapan-zona-waktu-di-dunia-hingga-usulan-penghapusannya?page=all">disini</a>.

#### <i><b><span style='color:#55679C'>Quest</span> : Tampilkan semua kolom pada tabel f_detail_transaksi dan tambahkan kolom data_collection_time sebagai kolom metadata waktu pengambilan data</b></i><br>

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
  *,
  TIMESTAMP(DATETIME(CURRENT_TIMESTAMP(), "Asia/Jakarta")) AS data_collection_time
FROM `dqlab-9876543.dq_pizza.f_detail_transaksi`

Executing query with job ID: dd155a4a-f5dd-42bd-a0db-b39aa75c0f05
Query executing: 1.48s
Job ID dd155a4a-f5dd-42bd-a0db-b39aa75c0f05 successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,order_id,order_details_id,customer_id,order_maker_id,pizza_name,category,size,price,production_cost,quantity,order_date,order_time,completion_time,is_complain,complain_detail,data_collection_time
0,TRX000734,TRXD001645,CUST003549,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,01/12/2024,20:00:30,20:27:49,0,None,2025-10-30 20:28:42.097540+00:00
1,TRX004103,TRXD009366,CUST006714,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,03/10/2024,19:10:04,19:39:33,0,None,2025-10-30 20:28:42.097540+00:00
2,TRX001914,TRXD004305,CUST008674,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,02/01/2024,19:24:03,19:36:58,0,None,2025-10-30 20:28:42.097540+00:00
3,TRX017344,TRXD039387,CUST004020,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,10/22/2024,13:20:05,13:39:42,0,None,2025-10-30 20:28:42.097540+00:00
4,TRX019817,TRXD045082,CUST003570,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,12/03/2024,16:40:44,17:07:19,0,None,2025-10-30 20:28:42.097540+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48618,TRX016165,TRXD036640,CUST005956,EMP000756,THE BRIE CARRE PIZZA,supreme,S,191565.0,170505.0,1,09/29/2024,21:05:44,21:33:24,0,None,2025-10-30 20:28:42.097540+00:00
48619,TRX008475,TRXD019280,CUST004817,EMP000756,THE BRIE CARRE PIZZA,supreme,S,191565.0,170505.0,1,05/22/2024,16:56:23,17:20:01,0,None,2025-10-30 20:28:42.097540+00:00
48620,TRX001581,TRXD003564,CUST009381,EMP000756,THE GREEK PIZZA,CLASSIC,XXL,291195.0,260738.0,1,01/27/2024,16:28:38,16:54:32,0,None,2025-10-30 20:28:42.097540+00:00
48621,TRX016753,TRXD037944,CUST004945,EMP000756,THE GREEK PIZZA,CLASSIC,XXL,291195.0,260738.0,1,10/10/2024,17:54:55,18:06:01,0,None,2025-10-30 20:28:42.097540+00:00


## <b>C.2. <span style='color:#0B2F9F'><code>Mengubah String menjadi Tanggal</code></span></b>

Adakalanya format tanggal yang disimpan pada database tidak disimpan dalam format date yang baku (<a href="https://www.iso.org/iso-8601-date-and-time-format.html">ISO 8601</a>) sehingga perlu penyesuaian. Gunakan : <br><br>
<table border="1">
    <thead>
        <tr>
            <th>Fungsi</th>
            <th>Deskripsi</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td><code>PARSE_DATE(format, string_expression)</code></td>
            <td>Mengubah String menjadi Date</td>
        </tr>
        <tr>
            <td><code>FORMAT_DATE(format, date_expression)</code></td>
            <td>Mengubah Date menjadi String</td>
        </tr>
    </tbody>
</table><br>

Untuk detail format tanggal akses link berikut https://docs.cloud.google.com/bigquery/docs/reference/standard-sql/format-elements

#### <i><b><span style='color:#55679C'>Quest</span> : Tampilkan order_date dan order_date_iso pada tabel f_detail_transaksi. Dimana kolom order_date merupakan kolom dengan data asli dan order_date_iso merupakan kolom order_date yang diubah formatnya menjadi YYYY-MM-DD sesuai ISO 8601!</b></i><br>

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
  order_date,
  PARSE_DATE('%m/%d/%Y', order_date) AS order_date_iso
FROM `dqlab-9876543.dq_pizza.f_detail_transaksi`

Executing query with job ID: e354cdb1-4c4c-461b-8f27-f3a49418b9e9
Query executing: 1.28s
Job ID e354cdb1-4c4c-461b-8f27-f3a49418b9e9 successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,order_date,order_date_iso
0,01/12/2024,2024-01-12
1,03/10/2024,2024-03-10
2,02/01/2024,2024-02-01
3,10/22/2024,2024-10-22
4,12/03/2024,2024-12-03
...,...,...
48618,11/27/2024,2024-11-27
48619,11/27/2024,2024-11-27
48620,11/27/2024,2024-11-27
48621,11/27/2024,2024-11-27


## <b>C.3. <span style='color:#0B2F9F'><code>Menambahkan atau Mengurangi Tanggal</code></span></b>

<table border="1">
    <thead>
        <tr>
            <th>Fungsi</th>
            <th>Deskripsi</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td><code>DATE_ADD(tanggal, INTERVAL jumlah_penambahan satuan_waktu)</code></td>
            <td>Menambahkan Tanggal pada Interval Tertentu</td>
        </tr>
        <tr>
            <td><code>DATE_SUB(tanggal, INTERVAL jumlah_penambahan satuan_waktu)</code></td>
            <td>Mengurangi Tanggal pada Interval Tertentu</td>
        </tr>
        <tr>
            <td><code>DATE_DIFF(end_date, start_date, date_part)</code></td>
            <td>Menghitung selisih tanggal</td>
        </tr>
    </tbody>
</table><br>

Sebagai contoh misalkan hari ini tanggal 30 Oktober 2025. Ingin diperoleh tanggal tepat 3 bulan yang lalu maka dapat digunakan sintaks `DATE_SUB(DATE('2025-10-30'), INTERVAL 3 MONTH)` maka akan diperoleh tanggal 30 Juli 2025.



#### <i><b><span style='color:#55679C'>Quest</span> : Manajer ingin melakukan analisa demografi customer, pada tabel customers tampilkan customer_id, customer_name, gender dan age-nya per hari ini!</b></i><br>

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT * FROM `dqlab-9876543.dq_pizza.customers`

Executing query with job ID: ad093379-5609-4d5e-aee6-6e9919840ec2
Query executing: 1.11s
Job ID ad093379-5609-4d5e-aee6-6e9919840ec2 successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,customer_id,customer_name,gender,birth_date,phone_number,email
0,CUST006458,Victoria Hutapea,F,1983-01-12,+62 (184) 333 2261,victoria.hutapea@hotmail.com
1,CUST005844,Dewi Mayasari,F,1986-05-19,(0409) 918-0204,dewi.mayasari@gmail.com
2,CUST001212,Ani Habibi,F,1981-07-31,+62 (888) 571 0613,ani.habibi@gmail.com
3,CUST003654,Oliva Pratiwi,F,1990-08-05,(0522) 928-9657,oliva.pratiwi@gmail.com
4,CUST003857,Kiandra Padmasari,F,2000-08-21,0865695318,kiandra.padmasari@gmail.com
...,...,...,...,...,...,...
132,CUST005303,Dwi Saefullah,M,1997-01-05,+62-0909-921-0282,dwi.saefullah@hotmail.com
133,CUST000968,Putu Wijaya,M,1997-06-24,0800694086,putu.wijaya@hotmail.com
134,CUST008990,Rangga Nababan,M,2007-01-04,+62-0992-496-9678,rangga.nababan@gmail.com
135,CUST009522,Cakrawangsa Hutagalung,M,1989-03-25,+62 (39) 581-0404,cakrawangsa.hutagalung@gmail.com


In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
  customer_id,
  customer_name,
  gender,
  birth_date,
  DATE_DIFF(CURRENT_DATE(), birth_date, YEAR) AS age
FROM `dqlab-9876543.dq_pizza.customers`

Executing query with job ID: 48be4a3e-9b7e-4abf-aa3f-994d751052d8
Query executing: 0.62s
Job ID 48be4a3e-9b7e-4abf-aa3f-994d751052d8 successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,customer_id,customer_name,gender,birth_date,age
0,CUST006458,Victoria Hutapea,F,1983-01-12,42
1,CUST005844,Dewi Mayasari,F,1986-05-19,39
2,CUST001212,Ani Habibi,F,1981-07-31,44
3,CUST003654,Oliva Pratiwi,F,1990-08-05,35
4,CUST003857,Kiandra Padmasari,F,2000-08-21,25
...,...,...,...,...,...
132,CUST005303,Dwi Saefullah,M,1997-01-05,28
133,CUST000968,Putu Wijaya,M,1997-06-24,28
134,CUST008990,Rangga Nababan,M,2007-01-04,18
135,CUST009522,Cakrawangsa Hutagalung,M,1989-03-25,36


## <b>C.5. <span style='color:#0B2F9F'><code>Mendapatkan Bagian pada Tanggal</code></span></b>

Mengambil bagian tertentu dari tipe data tanggal, waktu, atau timestamp terkadang sangat perlu dilakukan untuk analisa data. Jika ingin didapatkan elemen tertentu, seperti tahun, bulan, hari, atau jam, dari nilai tanggal atau waktu gunakan fungsi berikut<br><br>

<code>EXTRACT(bagian FROM tanggal)</code>
<br><br>

Bagian yang bisa diambil pada tanggal diantaranya adalah
<table border="1">
    <thead>
        <tr>
            <th>Bagian</th>
            <th>Deskripsi</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td><code>YEAR</code></td>
            <td>Mendapatkan Tahun pada Tanggal</td>
        </tr>
        <tr>
            <td><code>MONTH</code></td>
            <td>Mendapatkan Bulan pada Tanggal</td>
        </tr>
        <tr>
            <td><code>DAY</code></td>
            <td>Mendapatkan Hari (1-31) pada Tanggal</td>
        </tr>
        <tr>
            <td><code>WEEK</code></td>
            <td>Mendapatkan Minggu ke- pada Tanggal</td>
        </tr>
         <tr>
            <td><code>QUARTER</code></td>
            <td>Mendapatkan Kuartal pada Tanggal</td>
        </tr>
        <tr>
            <td><code>HOUR</code></td>
            <td>Mendapatkan Jam pada Waktu</td>
        </tr>
        <tr>
            <td><code>MINUTE</code></td>
            <td>Mendapatkan Menit pada Waktu</td>
        </tr>
        <tr>
            <td><code>SECOND</code></td>
            <td>Mendapatkan Detik pada Waktu</td>
        </tr>
    </tbody>
</table>
<br>

Namun jika menginginkan bagian yang lebih detail seperti Nama Bulan atau Nama Hari pada tanggal tertentu atau menampilkan format tanggal sesuai dengan keinginan dapat menggunakan sintaks `FORMAT_DATE(format_element, tanggal)`. <br><br>

<table border="1">
    <thead>
        <tr>
            <th>Sintaks</th>
            <th>Deskripsi</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td><code>FORMAT_DATE('%A', tanggal)</code></td>
            <td>Mendapatkan Nama Hari pada Tanggal Tertentu</td>
        </tr>
        <tr>
            <td><code>FORMAT_DATE('%B', tanggal)</code></td>
            <td>Mendapatkan Nama Bulan pada Tanggal Tertentu</td>
        </tr>
    </tbody>
</table>
<br>

Detail format_element dapat diakses dilaman <a href="https://cloud.google.com/bigquery/docs/reference/standard-sql/format-elements#format_elements_date_time"> berikut</a>.





#### <i><b><span style='color:#55679C'>Quest</span> : Hitung total transaksi dan total pendapatan yang diperoleh tiap bulan pada tahun 2024 di DQPizza! </b></i><br>

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
  EXTRACT(YEAR FROM PARSE_DATE('%m/%d/%Y', order_date)) AS year,
  EXTRACT(MONTH FROM PARSE_DATE('%m/%d/%Y', order_date)) AS month,
  FORMAT_DATE('%B', PARSE_DATE('%m/%d/%Y', order_date)) AS month_name,
  COUNT(DISTINCT order_id) AS total_transaksi,
  SUM(price * quantity) AS total_pendapatan
FROM `dqlab-9876543.dq_pizza.f_detail_transaksi`
GROUP BY
  1, 2, 3
ORDER BY
  1, 2

Executing query with job ID: af800b28-6808-48f5-b2e4-cb77cc92655f
Query executing: 0.61s
Job ID af800b28-6808-48f5-b2e4-cb77cc92655f successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,year,month,month_name,total_transaksi,total_pendapatan
0,2024,1,January,1845,565325730.0
1,2024,2,February,1685,527792760.0
2,2024,3,March,1840,570216510.0
3,2024,4,April,1799,556243605.0
4,2024,5,May,1853,578362275.0
5,2024,6,June,1773,552664620.0
6,2024,7,July,1935,587718990.0
7,2024,8,August,1841,553053825.0
8,2024,9,September,1661,520073055.0
9,2024,10,October,1646,518623560.0


# <b>D. <span style='color:#0B2F9F'><code>SUB-QUERY</code></span></b>
Subquery adalah sebuah query (pernyataan SQL) yang terletak di dalam query lain. Subquery dapat digunakan untuk mengambil data yang kemudian akan digunakan oleh query utama.<br><br>

#### <i><b><span style='color:#55679C'>Quest</span> : Pada tabel customers, hitung rata-rata umur customer dan bulatkan ke dua tempat desimal!</b></i>

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
  ROUND(AVG(DATE_DIFF(CURRENT_DATE(), birth_date, YEAR)), 2) AS age
FROM `dqlab-9876543.dq_pizza.customers`

Executing query with job ID: 218caf34-2c0d-49e3-a4f2-3b81bf39453f
Query executing: 0.63s
Job ID 218caf34-2c0d-49e3-a4f2-3b81bf39453f successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,age
0,34.04


#### <i><b><span style='color:#55679C'>Quest</span> : Pada tabel customers, tampilkan customer_id, age dan gendernya dimana umur customer tersebut lebih dari rata-rata umur customer secara keseluruhan!</b></i>

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
  customer_id,
  customer_name,
  DATE_DIFF(CURRENT_DATE(), birth_date, YEAR) AS age
FROM `dqlab-9876543.dq_pizza.customers`
WHERE
  DATE_DIFF(CURRENT_DATE(), birth_date, YEAR) > (SELECT AVG(DATE_DIFF(CURRENT_DATE(), birth_date, YEAR)) AS age FROM `dqlab-9876543.dq_pizza.customers`)

Executing query with job ID: 49219aab-9fb5-4540-9735-26e9bef39e9e
Query executing: 0.58s
Job ID 49219aab-9fb5-4540-9735-26e9bef39e9e successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,customer_id,customer_name,age
0,CUST006458,Victoria Hutapea,42
1,CUST005844,Dewi Mayasari,39
2,CUST001212,Ani Habibi,44
3,CUST003654,Oliva Pratiwi,35
4,CUST004438,Aurora Jailani,43
...,...,...,...
64,CUST005867,Jaswadi Suryono,37
65,CUST008255,Embuh Jailani,45
66,CUST002681,Nasab Halimah,47
67,CUST009522,Cakrawangsa Hutagalung,36


# <b>E. <span style='color:#0B2F9F'><code>Common Table Expression (CTE)</code></span></b>
CTE, atau Common Table Expression, adalah fitur dalam SQL yang memungkinkan Anda untuk mendefinisikan sebuah query sementara yang dapat digunakan dalam query lain. CTE membantu dalam membuat query lebih mudah dibaca dan dipelihara, serta memungkinkan penggunaan rekursi dalam query.

<img src="https://learnsql.com/blog/sql-common-table-expression-guide/cte_syntax_example.webp" width="50%">

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT * FROM `dqlab-9876543.dq_pizza.f_detail_transaksi`

Executing query with job ID: c19ad133-1f43-4e2b-beeb-7bee2d0e9368
Query executing: 2.75s
Job ID c19ad133-1f43-4e2b-beeb-7bee2d0e9368 successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,order_id,order_details_id,customer_id,order_maker_id,pizza_name,category,size,price,production_cost,quantity,order_date,order_time,completion_time,is_complain,complain_detail
0,TRX000734,TRXD001645,CUST003549,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,01/12/2024,20:00:30,20:27:49,0,None
1,TRX004103,TRXD009366,CUST006714,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,03/10/2024,19:10:04,19:39:33,0,None
2,TRX001914,TRXD004305,CUST008674,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,02/01/2024,19:24:03,19:36:58,0,None
3,TRX017344,TRXD039387,CUST004020,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,10/22/2024,13:20:05,13:39:42,0,None
4,TRX019817,TRXD045082,CUST003570,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,12/03/2024,16:40:44,17:07:19,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48618,TRX016165,TRXD036640,CUST005956,EMP000756,THE BRIE CARRE PIZZA,supreme,S,191565.0,170505.0,1,09/29/2024,21:05:44,21:33:24,0,None
48619,TRX008475,TRXD019280,CUST004817,EMP000756,THE BRIE CARRE PIZZA,supreme,S,191565.0,170505.0,1,05/22/2024,16:56:23,17:20:01,0,None
48620,TRX001581,TRXD003564,CUST009381,EMP000756,THE GREEK PIZZA,CLASSIC,XXL,291195.0,260738.0,1,01/27/2024,16:28:38,16:54:32,0,None
48621,TRX016753,TRXD037944,CUST004945,EMP000756,THE GREEK PIZZA,CLASSIC,XXL,291195.0,260738.0,1,10/10/2024,17:54:55,18:06:01,0,None


**Quest : Siapakah customer yang paling banyak spending money di DQPizza?**

In [ ]:
%%bigquery --project {project_id} --verbose

-- Using a Common Table Expression (CTE) to find the customer with the highest spending
WITH CustomerSpending AS (
    SELECT
        customer_id,
        SUM(price * quantity) AS total_spent
    FROM
        `dqlab-9876543.dq_pizza.f_detail_transaksi`
    GROUP BY
        customer_id
)
SELECT
    customer_id,
    total_spent
FROM
    CustomerSpending
ORDER BY
    total_spent DESC
LIMIT 1

Executing query with job ID: 0b8c84e3-437a-4f2d-9be7-76e939bf0381
Query executing: 0.43s
Job ID 0b8c84e3-437a-4f2d-9be7-76e939bf0381 successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,customer_id,total_spent
0,CUST006333,237372120.0


In [ ]:
%%bigquery --project {project_id} --verbose

WITH temp_tbl AS (
  SELECT
    order_id,
    customer_id,
    ROUND(SUM(((price - production_cost) * quantity)), 2) AS profit_before_tax
  FROM `dqlab-9876543.dq_pizza.f_detail_transaksi`
  WHERE order_details_id IS NOT NULL
  GROUP BY
    order_id, customer_id
)
  SELECT
    order_id,
    customer_id,
    profit_before_tax,
    (1-0.11) * profit_before_tax AS profit_after_tax,
    0.11 * profit_before_tax AS tax_paid
  FROM temp_tbl

Executing query with job ID: ec6af813-ffc6-41a7-a090-cf8399fff9d7
Query executing: 1.30s
Job ID ec6af813-ffc6-41a7-a090-cf8399fff9d7 successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,order_id,customer_id,profit_before_tax,profit_after_tax,tax_paid
0,TRX004103,CUST006714,88209.0,78506.01,9702.99
1,TRX003947,CUST009755,16038.0,14273.82,1764.18
2,TRX018599,CUST001735,74520.0,66322.80,8197.20
3,TRX011935,CUST004354,74034.0,65890.26,8143.74
4,TRX000387,CUST004307,157950.0,140575.50,17374.50
...,...,...,...,...,...
21341,TRX014773,CUST009740,93069.0,82831.41,10237.59
21342,TRX003295,CUST009740,24705.0,21987.45,2717.55
21343,TRX002839,CUST009740,22761.0,20257.29,2503.71
21344,TRX003352,CUST009740,49329.0,43902.81,5426.19


In [ ]:
%%bigquery --project {project_id} --verbose

WITH temp_tbl AS (
  SELECT
    customer_id,
    ROUND(SUM(((price - production_cost) * quantity)), 2) AS profit_before_tax
  FROM `dqlab-9876543.dq_pizza.f_detail_transaksi`
  WHERE order_details_id IS NOT NULL
  GROUP BY
    order_id, customer_id
)
  SELECT
    customer_id,
    profit_before_tax
  FROM temp_tbl
  WHERE profit_before_tax = (SELECT MAX(profit_before_tax) FROM temp_tbl)

Executing query with job ID: fb885d1a-f400-4da8-a4a7-a1fffe13fe23
Query executing: 0.62s
Job ID fb885d1a-f400-4da8-a4a7-a1fffe13fe23 successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,customer_id,profit_before_tax
0,CUST007150,747954.0


Data Source : <i>https://mavenanalytics.io/challenges/maven-pizza-challenge</i> (dengan modifikasi)


---

<br>
<a href="https://www.linkedin.com/in/danang-arya-saputra/"><img src="https://img.shields.io/badge/-© 2025 Danang Arya Saputra-417DAC?style=for-the-badge&logoColor=white"/></a>

<a href="https://dqlab.id/"><img src="https://dqlab.id/files/dqlab/cache/87e30118ebba5ec7d96f6ea8c9dcc10b_x_118_X_55.png" align="left" /></a>
